<a href="https://colab.research.google.com/github/astrovishalthakur/MachineLearning/blob/main/SklearnFeatures/Pipelining/WithPipeLine/TitanicWithPipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

In [3]:
df = pd.read_csv("titanic.csv")

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Let's Plan

# Pipeline processes

### Two columns has data missing, age and embark.
### 1. we need to do is fill that using simple imputer.
### 2. we will one hot encode sex and embark.
### 3. we will do scaling.
### 4. Feature selection (take best 5 feature)
### 5. Train model using Decision Tree.

In [5]:
df.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"], inplace=True)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=["Survived"]), df.Survived, test_size=0.2, random_state=42)

In [13]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
331,1,male,45.5,0,0,28.5000,S
733,2,male,23.0,0,0,13.0000,S
382,3,male,32.0,0,0,7.9250,S
704,3,male,26.0,1,0,7.8542,S
813,3,female,6.0,4,2,31.2750,S


In [14]:
y_train.sample(5)

554    1
516    1
431    1
95     0
46     0
Name: Survived, dtype: int64

In [19]:
# imputation transformer

trf1 = ColumnTransformer(
    [
     ("impute_age", SimpleImputer(), [2]), # here we are passing [2] which means column at index 2, it is done because return of transformation is numpy array which doesn't have column name but rather index number.
     ("impute_embarked", SimpleImputer(strategy="most_frequent"), [6])
    ],
    remainder='passthrough' # to avoid dropping remaining columns
)
# during pipeline creation always use index rather then column names.
# as one transformation returns numpy array, 
# when fed to next transformation as input, it wouldn't know the column names anymore, but rather index.

In [18]:
# one hot encoding
trf2 = ColumnTransformer(
    [
     ("ohe_sex_embarked", OneHotEncoder(sparse=False, handle_unknown="ignore"), [1, 6]) # not dropping first cause we will use decision tree.
    ],
    remainder="passthrough"
)

In [20]:
# Scaling
trf3 = ColumnTransformer(
    [
     ("scale", MinMaxScaler(), slice(0, 10)) # slice(0, 10) means apply minmaxscaler to all 10 cols. i.e. 0 to 9
     # when we will apply onehotencoder on sex and embark, 2 new cols will be created from sex
     # and 3 new cols from embark.
     # previous sex and embark will be removed and new will be added.
     # so initially data had 7 cols,
     # 2 got removed, sex and embark. New no. of cols = 7-2 = 5
     # new 5 will be added, 2 for sex, 3 for embark. New no. of cols = 5+2+3 = 10. i.e. slice(0, 10)
    ]
)

In [21]:
# Feature Scaling

trf4 = SelectKBest(score_func=chi2, k=8)# k=8 means use top 8 imp cols out of 10

In [22]:
# train model

trf5 = DecisionTreeClassifier()

# Create Pipeline

In [23]:
pipe = Pipeline(
    [
    ("trf1", trf1),
    ("trf2", trf2),
    ("trf3", trf3),
    ("trf4", trf4),
    ("trf5", trf5)
    ]
)

# Pipeline Vs make_pipeline

Pipeline requires naming of steps, make_pipeline does not. <br>
(Same applies to ColumnTransformer vs make_column_transformer)

In [24]:
# Alternate Syntax

pipe2 = make_pipeline(trf1, trf2, trf3, trf4, trf5)

In [25]:
# train

pipe.fit(X_train, y_train)

# if we don't have last pipeline transformer as model, we can just call fit_transform.
# we are only calling fit here so that we can call predict later.

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),


In [28]:
# Display pipeline using tables.
# run this code to see this.

from sklearn import set_config

set_config(display="diagram")

In [29]:
pipe2.fit(X_train, y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('columntransformer-2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('columntransformer-3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7fba67d09f80>)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

# Explore Pipeline

In [30]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x7fba67d09f80>),
 'trf5': DecisionTreeClassifier()}

In [31]:
pipe.named_steps['trf1'].transformers_

[('impute_age', SimpleImputer(), [2]),
 ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder', 'passthrough', [0, 1, 3, 4, 5])]

In [33]:
pipe.named_steps["trf1"].transformers_[0][1]

SimpleImputer()

In [36]:
pipe.named_steps["trf1"].transformers_[0][1].statistics_
# this is mean value calculated by simple imputer

array([29.49884615])

In [37]:
pipe.named_steps["trf1"].transformers_[1][1]

SimpleImputer(strategy='most_frequent')

In [38]:
pipe.named_steps["trf1"].transformers_[1][1].statistics_
# this is most frequent value in data. This was used by simpleimputer to fill missing values.

array(['S'], dtype=object)

In [40]:
# Predict

y_pred_1 = pipe.predict(X_test) 
y_pred_2 = pipe2.predict(X_test)

In [43]:
[y_pred_1 == y_pred_2] == False

False

In [44]:
y_pred_1

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0])

In [46]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_1)
# the reason accuracy is low as compare to 80 to 85%, because we removed some columns in the starting.

0.6256983240223464

# CrossValidation using Pipeline

In [47]:
# cross validation using cross_val_score

from sklearn.model_selection import cross_val_score

cross_val_score(pipe, X_train, y_train, cv=5, scoring="accuracy").mean()

# by cv = 5 means do train_test_split 5 times, then calculate accuracy each time and give mean of that.

0.6391214419383433

# GridSearch using Pipeline

In [48]:
# gridsearchcv

params = {
    "trf5__max_depth":[1, 2, 3, 4, 5, None]
    # "trf5__max_depth, this is not variable name but writing format
    # i.e. "transformername__parametername"
    # tranformer name + double underscore + parameter name
}

In [49]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring="accuracy")
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x7fba67d09f80>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [50]:
grid.best_score_

0.6391214419383433

In [51]:
grid.best_params_

{'trf5__max_depth': 2}

# Exporting the pipeline

In [52]:
# export
import pickle
pickle.dump(pipe, open("pipe.pkl", "wb"))